In [31]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


In [51]:
df = pd.read_csv('rnn_df.csv')
df.head()

,Unnamed: 0,CurrentStatus,processed
0,0,Complete,"['call', '7', 'attempt', '1', 'message', 'left..."
1,1,Complete,"['welcome', 'call', 'advised', 'call', 'record..."
2,2,Complete,"['welcome', 'call', 'attempt', '1', 'engaged',..."
3,3,Complete,"['welcome', 'call', 'member', 'consented', 'pr..."
4,4,Ceased,"['welcome', 'call', 'attended', 'intended', 'r..."


In [52]:
df = df.drop(df[df['CurrentStatus']=='Enrolled'].index)

In [53]:
df['CurrentStatus'] = np.where(df['CurrentStatus']=='Complete', 1, 0)
X_train, X_test, y_train, y_test = train_test_split(df['processed'],
                                                    df['CurrentStatus'], test_size=0.2)

In [54]:
X = df['processed']

In [55]:
y_train

1196    1
748     0
676     0
382     0
402     1
       ..
510     1
561     1
35      1
1003    1
878     1
Name: CurrentStatus, Length: 944, dtype: int64

In [56]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [57]:
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [58]:
#make each seq of the same length; if short it will pad, if long it will cut down
X_train_seq_padded = pad_sequences(X_train_seq, 50)
X_test_seq_padded = pad_sequences(X_test_seq, 50)

In [59]:
import keras.backend as K
from keras.layers import Dense, Embedding, LSTM
from keras.models import Sequential

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

In [60]:
#should tune the number of output dimentionality
model = Sequential()

model.add(Embedding(len(tokenizer.index_word)+1, 32))
model.add(LSTM(32, dropout=0, recurrent_dropout=0))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 32)          384384    
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_4 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 33        
Total params: 393,793
Trainable params: 393,793
Non-trainable params: 0
_________________________________________________________________


In [61]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', precision_m, recall_m])

In [62]:

# Fit the RNN
history = model.fit(X_train_seq_padded, y_train, 
                    batch_size=32, epochs=10,
                    validation_data=(X_test_seq_padded, y_test))

Epoch 1/10
30/30 [==============================] - 4s 59ms/step - loss: 0.6580 - accuracy: 0.7651 - precision_m: 0.7887 - recall_m: 0.9370 - val_loss: 0.4198 - val_accuracy: 0.8475 - val_precision_m: 0.8398 - val_recall_m: 1.0000
Epoch 2/10
30/30 [==============================] - 1s 33ms/step - loss: 0.4557 - accuracy: 0.8000 - precision_m: 0.8002 - recall_m: 1.0000 - val_loss: 0.3085 - val_accuracy: 0.8686 - val_precision_m: 0.8617 - val_recall_m: 1.0000
Epoch 3/10
30/30 [==============================] - 1s 34ms/step - loss: 0.2775 - accuracy: 0.8505 - precision_m: 0.8525 - recall_m: 0.9821 - val_loss: 0.2936 - val_accuracy: 0.8602 - val_precision_m: 0.9302 - val_recall_m: 0.9178
Epoch 4/10
30/30 [==============================] - 1s 32ms/step - loss: 0.1921 - accuracy: 0.9200 - precision_m: 0.9423 - recall_m: 0.9559 - val_loss: 0.3609 - val_accuracy: 0.8941 - val_precision_m: 0.9332 - val_recall_m: 0.9560
Epoch 5/10
30/30 [==============================] - 1s 32ms/step - loss: 0.1